# Obtendo dados de Pegadas de Edificios a partir do City2Graph

In [ ]:
# Geospatial data processing
import geopandas as gpd

# The star of the show: city2graph for transportation network analysis
import city2graph as c2g
import city2graph.data as c2g_data

print("All dependencies loaded successfully!")
print(f"city2graph version: {c2g.__version__ if hasattr(c2g, '__version__') else 'development'}")

All dependencies loaded successfully!
city2graph version: 0.1.6


# 3. Baixando e Carregando dados Overture Maps

## 3.1 Essa etapa só deve ser executada, caso queira ter acesso aos dados mais recentes da Overture.

In [ ]:
# Download data from Overture Maps (uncomment to download fresh data)
# This downloads building footprints, road segments, and connectors for Teresina city centre

bbox_ae_3km = [-42.863200,-5.259269, -42.626205, -4.929381]

path_dados_intermediario = "dados/overturemaps/intermediarios/"
path_dados_finais = "/dados/overturemaps/finais/"
types = ["building"]

overture_data = c2g_data.load_overture_data(
    area=bbox_ae_3km, # Área da zona leste de Teresina
    types=types,
    output_dir=path_dados_intermediario,
    prefix="teresina_bbox_ae3km_",
    save_to_file=True,
    return_data=True
)

gdf_building = overture_data['building']

print(f"O download dos dados {types} foi concluído.")

O download dos dados ['building'] foi concluído.


## 1.2 Carregando os dados

In [2]:
gdf_building = gpd.read_file("./dados/overturemaps/intermediarios/teresina_bbox_ae3km_building.geojson")

In [3]:
gdf_building_utm = gdf_building.to_crs(epsg=31983)

# 2. Carregando dados da Área de Estudo

In [4]:
# Carregar arquivos de referência da pesquisa
zu = gpd.read_file("dados/zona_urbana/zu.geojson")
zu_utm = zu.to_crs(epsg=31983)

zu_buffer3km = gpd.read_file("dados/zona_urbana/zu_buffer3km.geojson")
zu_buffer3km_utm = zu_buffer3km.to_crs(epsg=31983)

In [5]:
# Seleciona os segmentos que INTERSECIONAM com a área de estudo 3km.
print(f"Total de segmentos antes da operação espacial: interseção: Segmentos = {len(gdf_building_utm)}")

gdf_building_utm_zu = gdf_building_utm.sjoin(
    zu_utm, 
    how='inner', 
    predicate='intersects',
)

print(f"Total de segmentos antes da operação espacial: interseção: Segmentos = {len(gdf_building_utm_zu)} ({len(gdf_building_utm) - len(gdf_building_utm_zu)})")

Total de segmentos antes da operação espacial: interseção: Segmentos = 445720
Total de segmentos antes da operação espacial: interseção: Segmentos = 346692 (99028)


In [6]:
# Seleciona os segmentos que INTERSECIONAM com a área de estudo 3km.
print(f"Total de segmentos antes da operação espacial: interseção: Segmentos = {len(gdf_building_utm)}")

gdf_building_utm_zu_buffer3km = gdf_building_utm.sjoin(
    zu_buffer3km_utm, 
    how='inner', 
    predicate='intersects',
)

print(f"Total de segmentos antes da operação espacial: interseção: Segmentos = {len(gdf_building_utm_zu_buffer3km)} ({len(gdf_building_utm) - len(gdf_building_utm_zu_buffer3km)})")

Total de segmentos antes da operação espacial: interseção: Segmentos = 445720
Total de segmentos antes da operação espacial: interseção: Segmentos = 362934 (82786)


In [7]:
# Salvando em geoparquet
gdf_building_utm_zu.to_parquet(
    "dados/overturemaps/finais/building_zu_utm.parquet"
)

In [8]:
# Salvando em geoparquet
gdf_building_utm_zu_buffer3km.to_parquet(
    "dados/overturemaps/finais/building_zu3km_utm.parquet"
)